In [1]:

from textwrap import dedent

from pydantic import BaseModel, Field

from xverify import GuidedSchema


In [8]:
class Think(BaseModel):
    """
    Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.
    """

    thought: str = Field(
        ...,
        description="A thought to think about.",
    )

schema = GuidedSchema(Think)
print(schema.doc)
print("\n" + "*" * 100 + "\n")
print(schema.gbnf)

xml = """
<Think>
<thought>
Let me think about this step by step...
</thought>
</Think>
"""

schema.parse(xml)

Output Model: Think
  Description: Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.
  Output Fields:
    thought (str):
        Description: A thought to think about.

****************************************************************************************************

root ::= grammar-models
string ::= nl [^\n<] ([^<])*
boolean ::= nl "true" | nl "false"
integer ::= nl [0-9]+
float ::= nl "-"? [0-9]+ ("." [0-9]+)?
null ::= nl "null"
nl ::= "\n"
Think ::=  "<Think>" nl "<thought>" string nl "</thought>" nl "</Think>"
grammar-models ::= Think


Think(thought='Let me think about this step by step...')

In [18]:

class ThinkAirline(BaseModel):
    """
    Before taking any action or responding to the user after receiving tool results, use the think tool as a scratchpad to:
    - List the specific rules that apply to the current request
    - Check if all required information is collected
    - Verify that the planned action complies with all policies
    - Iterate over tool results for correctness

    """

    class Config:
        json_schema_extra = {
            "example": [
                dedent("""\
                User wants to cancel flight ABC123
                - Need to verify: user ID, reservation ID, reason
                - Check cancellation rules:
                  * Is it within 24h of booking?
                  * If not, check ticket class and insurance
                - Verify no segments flown or are in the past
                - Plan: collect missing info, verify rules, get confirmation
                """),
                dedent("""\
                User wants to book 3 tickets to NYC with 2 checked bags each
                - Need user ID to check:
                  * Membership tier for baggage allowance
                  * Which payments methods exist in profile
                - Baggage calculation:
                  * Economy class x 3 passengers
                  * If regular member: 1 free bag each → 3 extra bags = $150
                  * If silver member: 2 free bags each → 0 extra bags = $0
                  * If gold member: 3 free bags each → 0 extra bags = $0
                - Payment rules to verify:
                  * Max 1 travel certificate, 1 credit card, 3 gift cards
                  * All payment methods must be in profile
                  * Travel certificate remainder goes to waste
                - Plan:
                1. Get user ID
                2. Verify membership level for bag fees
                3. Check which payment methods in profile and if their combination is allowed
                4. Calculate total: ticket price + any bag fees
                5. Get explicit confirmation for booking
                """),
            ],
        "description": "testing",
        "title": "think"
        }

    thought: str = Field(
        ...,
        description="A thought to think about.",
    )


schema = GuidedSchema(ThinkAirline)
print(schema.doc)
print("\n" + "*" * 100 + "\n")
print(schema.gbnf)


Output Model: ThinkAirline
  Description: Before taking any action or responding to the user after receiving tool results, use the think tool as a scratchpad to:
    - List the specific rules that apply to the current request
    - Check if all required information is collected
    - Verify that the planned action complies with all policies
    - Iterate over tool results for correctness
  Output Fields:
    thought (str):
        Description: A thought to think about.
  Expected Example Output for ThinkAirline:
    [
      "User wants to cancel flight ABC123\n- Need to verify: user ID, reservation ID, reason\n- Check cancellation rules:\n  * Is it within 24h of booking?\n  * If not, check ticket class and insurance\n- Verify no segments flown or are in the past\n- Plan: collect missing info, verify rules, get confirmation\n",
      "User wants to book 3 tickets to NYC with 2 checked bags each\n- Need user ID to check:\n  * Membership tier for baggage allowance\n  * Which payments meth

In [4]:
from xverify import XMLToolUse


def think(
    thought: str # A thought to think about.
):
    """
    Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.
    """
    return "thought saved"


think_model = XMLToolUse[think]
schema = GuidedSchema(think_model)

print(schema.gbnf)
print("\n" + "*" * 100 + "\n")
print(schema.doc)

root ::= grammar-models
string ::= nl [^\n<] ([^<])*
boolean ::= nl "true" | nl "false"
integer ::= nl [0-9]+
float ::= nl "-"? [0-9]+ ("." [0-9]+)?
null ::= nl "null"
nl ::= "\n"
think ::=  "<think>" nl "<thought>" string nl "</thought>" nl "</think>"
grammar-models ::= think

****************************************************************************************************

Output Model: think
  Description: Use the tool to think about something. It will not obtain new information or change the database, but just append the thought to the log. Use it when complex reasoning or some cache memory is needed.
  Output Fields:
    thought (str):
        Description: A thought to think about.


In [5]:

class Reason_and_Act(BaseModel):
    scratchpad: str = Field(
        ...,
        description="Information from the Observation useful to answer the question",
    )
    reasoning: str = Field(
        ...,
        description="It describes your thoughts about the question you have been asked",
    )

schema = GuidedSchema(Reason_and_Act)
print(schema.doc)
print("\n" + "*" * 100 + "\n")
print(schema.gbnf)




Output Model: Reason_and_Act
  Output Fields:
    scratchpad (str):
        Description: Information from the Observation useful to answer the question
    reasoning (str):
        Description: It describes your thoughts about the question you have been asked

****************************************************************************************************

root ::= grammar-models
string ::= nl [^\n<] ([^<])*
boolean ::= nl "true" | nl "false"
integer ::= nl [0-9]+
float ::= nl "-"? [0-9]+ ("." [0-9]+)?
null ::= nl "null"
nl ::= "\n"
Reason_and_Act ::=  "<Reason_and_Act>" nl "<scratchpad>" string nl "</scratchpad>" nl "<reasoning>" string nl "</reasoning>" nl "</Reason_and_Act>"
grammar-models ::= Reason_and_Act
